In [7]:

import os
import PySimpleGUI as sg
from pathlib import Path
import win32com.client
#from pptx.enum.shapes import MSO_SHAPE_TYPE
from win32api import RGB
import shutil
# from pptx import Presentation

def create_empty_pptx(path_file, list_title_create):
    """Создает пустую презу. С нужными заголовками"""
    pptx = win32com.client.Dispatch("PowerPoint.Application")
    empty_pres = pptx.Presentations.Add(1)
    count_empy_slide = len(list_title_create)


    for n_sl in range(1, count_empy_slide + 1):
        title_name = list_title_create[n_sl-1] + '-*-'
        empty_pres.Slides.Add(n_sl,11)  #i номер слайда, 11 шаблон слайд где есть только заголовок.
        

        slide = empty_pres.Slides(n_sl)
        title = slide.Shapes.Title
        title.name = 'Заголовок-*-'
        title.TextFrame.TextRange.Text = title_name
        slide.Shapes(1).TextFrame.TextRange.Font.Size = 20
        title.Width = 200 
        title.Left = -200
        title.Top = 0
        title.Height = 40

    empty_pres.SaveAs(path_file)
    empty_pres.Close()

    pptx.Quit()

def create_title_shape(slide, name_tit, left, top, width, Height):
    """создает фигуру заголовка, для последующего добавления в слайды"""
    shape_figure = 5
    shape = slide.Shapes.AddShape(Type=shape_figure, Left=left, Top=top, Width=width, Height=Height)
    shape.Fill.ForeColor.RGB = RGB(175, 185, 205)
    shape.Line.Visible = 0 # no border

    shape.name = name_tit + '-*-'
    shape.TextFrame.TextRange.Text = name_tit
    shape.TextFrame.TextRange.Font.Color.RGB = RGB(18, 43, 74)
    shape.TextFrame.TextRange.Font.Size = 11
    
    return left, top

def create_template(path_file, titles_entered):
    """Создание шаблона"""

    pptx = win32com.client.Dispatch("PowerPoint.Application")
    pptx.Visible = True

    empty_pres = pptx.Presentations.Add(1)
    count_reapet_head = 3
    list_title = []
    for title in titles_entered:
        for _ in range(count_reapet_head):
            list_title.append(title)
        
    count_empy_slide = len(list_title)

    slide = empty_pres.Slides.Add(1,12)
    #Добавляю стандартные фигруы:
    shape = slide.Shapes.AddShape(Type=1, Left=0, Top=0, Width=empty_pres.PageSetup.SlideWidth, Height=40)
    shape.Fill.ForeColor.RGB = RGB(18, 43, 74)  
    shape.name = 'Верхний фон'

    shape = slide.Shapes.AddShape(Type=34, Left=890, Top=10, Width=30, Height=30) #129
    shape.Fill.ForeColor.RGB = RGB(175, 185, 205) #RGB(255, 255, 255)
    shape.name = 'Стрелка предыдущий слайд'
    shape = slide.Shapes.AddShape(Type=33, Left=930, Top=10, Width=30, Height=30)
    shape.Fill.ForeColor.RGB = RGB(175, 185, 205)
    shape.name = 'Стрелка следующий слайд'

    width = 70
    Height = 30
    left = 10
    top = 150 

    titles_entered.append('..._')
    titles_entered.append('_...')
    
    for titl in titles_entered:
        create_title_shape(slide=slide, name_tit=titl, left=left, top=top, width=width, Height=Height)
        left += width + 10 
        
        if left > 960 - width:
            left = 10
            top += 60

    empty_pres.SaveAs(path_file)
    empty_pres.Close()

    pptx.Quit()

    
def timpl_and_empty(path_save, titles_introducted):
    # path_empty = ''
    path_save = Path(path_save)
    file_name_empty = path_save.joinpath('Пустой файл с проставленной структурой заголовков.pptx')
    file_name_templ = path_save.joinpath('Шаблон из заголовков для презы.pptx')

    titles_list = []
    list_title_create = []
    count_reapet_head = 3
    for title in titles_introducted.split(','):
        title_s = str(title).replace('\n', '')
        title_s = title_s.strip()
        titles_list.append(title_s)
        for _ in range(count_reapet_head):
            list_title_create.append(title_s)
            

    create_empty_pptx(path_file=file_name_empty, list_title_create=list_title_create)
    print('Создан пустой файл с заголовками для заполения')
    create_template(path_file=file_name_templ, titles_entered=titles_list)
    print('Создан шаблон для формирования шапки')
    print('Можно закрывать')
    


# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!добить эту функию

def create_empy_and_fill(path_save, file_copy, Nslides_and_titles):
    print('path_save', path_save)
    print('file_copy', file_copy)
    
    path_save = Path(path_save)
    file_name_empty_fill = path_save.joinpath('Пустой файл с проставленной структурой заголовков.pptx')
    file_name_template = path_save.joinpath('Шаблон для формирования презы.2.pptx')
    file_name_fill = path_save.joinpath('Заполненный файл.pptx')
    file_copy = Path(file_copy)
    print('path_save', path_save)
    print('file_copy', file_copy)
    print('file_name_empty_fill', file_name_empty_fill)
    
    
    dict_NSl_SlName = dict()
    list_name = []
    list_title_create = []
    for Nsl_name in Nslides_and_titles.split(','):
        n_slides, sl_name = Nsl_name.strip().split('=')
#         sl_name = sl_name.lower()
        n_sl_start, s_sl_end = n_slides.split('-')
        n_sl = [i for i in range(int(n_sl_start), int(s_sl_end)+1)]
        dict_NSl_SlName[sl_name] = n_sl
        list_name.append(sl_name)
        for _ in range(len(n_sl)):
            list_title_create.append(sl_name)
    print('dict_NSl_SlName', dict_NSl_SlName)
    create_empty_pptx(path_file=file_name_empty_fill, list_title_create=list_title_create)
    #Создали пустой с заголовками
    
    #Подлючаемся 
    pptx = win32com.client.Dispatch("PowerPoint.Application")
    presentation_slides_copy_of = pptx.Presentations.Open(str(file_copy), ReadOnly=True, WithWindow=False)
    presentation_slides_copy_in = pptx.Presentations.Open(str(file_name_empty_fill), ReadOnly=True, WithWindow=False)
    
    sl_paste = 1
    for name in list_name:
        list_num_slide = dict_NSl_SlName[name]
        first_slide = min(list_num_slide)
        for sl_copy in list_num_slide:
            print('sl_copy', sl_copy)
            print('sl_paste', sl_paste)
            presentation_slides_copy_of.Slides(sl_copy).Shapes.Range().Copy()
            presentation_slides_copy_in.Slides(sl_paste).Shapes.Paste().ZOrder(1)
            sl_paste += 1
            
            
    create_template(path_file=file_name_template, titles_entered=list_name)
            
            
    presentation_slides_copy_in.SaveAs(file_name_fill)
    presentation_slides_copy_of.Close()
    presentation_slides_copy_in.Close()
    pptx.Quit()
    print('Копирование выполнено. Можно закрывать')


right_click_menu = ['', ['Copy', 'Paste', 'Select All', 'Cut']]
    
layout = [#[sg.Input(default_text=path_tipl, key='-path_tipl_new-'), sg.FileBrowse(button_text='Расположение ШАБЛОНА', key='-path_tipl_new-'), sg.Button('сохранить папку сохранения',enable_events=True, key='-save_path_tipl-'), sg.Text('Выбрать расположение шаблона')],
        [sg.Text('Создание пуской презентации со структурой и шаблона шапки.', font='Helvetica 12')],
        [sg.Multiline(key='-title_nams-', right_click_menu=right_click_menu), sg.Text('Ввести: названия требуемых заголовков через запятую.')] ,
            [sg.Input(key='-path_tepl_and_empty-'), 
                                sg.FolderBrowse(button_text='Папка сохранения', key='-path_tepl_and_empty-'), 
                                sg.Text('Сохранение pptx: шаблна, и пусктого с заголовками')
                                ],
          [sg.Button('Формировать',enable_events=True, key='-Function_fimp_empy-', font='Helvetica 10')],
          [sg.Text('Создание пуской презентации со структурой и её заполнение из другой презентации', font='Helvetica 12')],
          [sg.Multiline(key='-Nslides_and_titles-', right_click_menu=right_click_menu), sg.Text('Ввести: промежуток_номеров_слайдов_в_исходной_през. = название_раздела,...')],
          [sg.Text('Пример: 1-3=Бюджет,4-7=Счета, ... ')],
          [sg.Input(key='-file_slides_copy_of2-'), sg.FileBrowse(button_text='Преза из которой', key='-file_slides_copy_of2-')],
          [sg.Input(key='-path_empty-'), 
                                            sg.FolderBrowse(button_text='Папка сохранения', key='-path_empty-'), 
                                            sg.Text('Сохранение pptx: шаблна, и пусктого с заголовками')
                                            ],
          [sg.Button('Формировать',enable_events=True, key='-Function_Nsl_titl-', font='Helvetica 10')],
          [sg.Output(size=(100,10))]
         ]

window = sg.Window('Добавление заголовков презентации по шаблону', layout)
mline1:sg.Multiline = window['-title_nams-']
mline2:sg.Multiline = window['-Nslides_and_titles-']

flag_do = True
while True:
    event, values = window.read()
    
    if event in (sg.WIN_CLOSED, 'Exit'):
        break
    # path_tipl_new = values['-path_tipl_new-']
    path_save_tepl_n_empty =  values['-path_tepl_and_empty-']
    titles_introducted = values['-title_nams-']    
    Nslides_and_titles = values['-Nslides_and_titles-']
#     file_slides_copy_of = values['-file_slides_copy_of-']
#     file_slides_copy_in = values['-file_slides_copy_in-']
    file_slides_copy_of2 = values['-file_slides_copy_of2-']
    path_empty = values['-path_empty-']


    path_true = False
    
    if event in (sg.WIN_CLOSED, 'Exit'):
        break
    if event == 'Select All':
        mline1.Widget.selection_clear()
        mline1.Widget.tag_add('sel', '1.0', 'end')
    elif event == 'Copy':
        try:
            text = mline1.Widget.selection_get()
            window.TKroot.clipboard_clear()
            window.TKroot.clipboard_append(text)
        except:
            print('Nothing selected')
    elif event == 'Paste':
        mline1.Widget.insert(sg.tk.INSERT, window.TKroot.clipboard_get())
    elif event == 'Cut':
        try:
            text = mline1.Widget.selection_get()
            window.TKroot.clipboard_clear()
            window.TKroot.clipboard_append(text)
            mline1.update('')
        except:
            print('Nothing selected')
    if event in (sg.WIN_CLOSED, 'Exit'):
        break
    if event == 'Select All':
        mline2.Widget.selection_clear()
        mline2.Widget.tag_add('sel', '1.0', 'end')
    elif event == 'Copy':
        try:
            text = mline2.Widget.selection_get()
            window.TKroot.clipboard_clear()
            window.TKroot.clipboard_append(text)
        except:
            print('Nothing selected')
    elif event == 'Paste':
        mline2.Widget.insert(sg.tk.INSERT, window.TKroot.clipboard_get())
    elif event == 'Cut':
        try:
            text = mline2.Widget.selection_get()
            window.TKroot.clipboard_clear()
            window.TKroot.clipboard_append(text)
            mline2.update('')
        except:
            print('Nothing selected')


    if event == '-Function_fimp_empy-':
        timpl_and_empty(path_save=path_save_tepl_n_empty, titles_introducted=titles_introducted)
      
    if event == '-Function_Nsl_titl-':
         create_empy_and_fill(path_save=path_empty, file_copy=file_slides_copy_of2, Nslides_and_titles=Nslides_and_titles)
            #create_empy_and_fill(path_save=path_save_tepl_n_empty, Nslides_and_titles=Nslides_and_titles)
window.close()
